Execute the Following SQL queries;

--Postgresql Connection Details:

In [ ]:
import psycopg2
import streamlit as st
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

import json
import toml

import sys
import os

# Load environment
try:
    with open("C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project1/Tennis_Data_SportRadar/config.json", "r") as f:
        config = json.load(f)

    env = config["environment"] # 'local' or 'cloud'
except FileNotFoundError:
    raise FileNotFoundError("⚠️ 'config.json' not found. Please check the path.")
except json.JSONDecodeError:
    raise ValueError("⚠️ 'config.json' is not a valid JSON file.")
except KeyError:
    raise KeyError("⚠️ 'environment' key not found in config.json.")

# Load secrets from your custom toml file
local_secrets = toml.load("C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project1/Tennis_Data_SportRadar/.pwd/pwd.toml")



# Append the directory where db_utils.py is located
sys.path.append(os.path.abspath('C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project1/Tennis_Data_SportRadar'))


from db_utils import connect_to_postgres, connect_to_sportradar

# Connect to default DB and create target DB if needed
my_db_connection = connect_to_postgres(env, local_secrets) #connect to existing database first

# Set autocommit so CREATE DATABASE works
my_db_connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

# Create cursor and check for database existence
mycursor = my_db_connection.cursor()
mycursor.execute("SELECT * FROM pg_database WHERE datname='sportradar'")
exists = mycursor.fetchone()

if not exists:
    mycursor.execute("CREATE DATABASE sportradar")   #create target DB
    print("Database 'sportradar' created")
else:
    print("Database 'sportradar' already exists")

mycursor.close()
my_db_connection.close()




# Reconnect to the newly ensured 'sportradar' database
my_db_connection = connect_to_sportradar(env, local_secrets)

my_db_connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

#I need to connect the db and the python:
mycursor = my_db_connection.cursor()

print("Connected to 'sportradar' database.")

Database 'sportradar' already exists
Connected to 'sportradar' database.


In [45]:
import pandas as pd

1.COMPETITIONS:

1) List all competitions along with their category name:

In [46]:
sql_query = """Select comp.competition_id, comp.competition_name,comp.competition_type,comp.competition_gender,
cat.categoryname 
from 
competition_table comp full join categories_table cat 
on cat.categoryid = comp.categoryid"""

#execute:
mycursor.execute(sql_query)
fetching_data = mycursor.fetchall()


In [47]:
#To fetch column names:
a = mycursor.description

# Create an empty list to store column names
column_names = []

# Loop through each tuple and extract the first item
for col in a:
    column_names.append(col[0])
    
# Create DataFrame
df = pd.DataFrame(fetching_data, columns=column_names)

# Display the DataFrame
print(df)

            competition_id                                   competition_name  \
0       sr:competition:620                                         Hopman Cup   
1       sr:competition:660                                     World Team Cup   
2       sr:competition:990                         ATP Challenger Tour Finals   
3      sr:competition:1207                  Championship International Series   
4      sr:competition:2100                                          Davis Cup   
...                    ...                                                ...   
6034  sr:competition:46999                                     UTR Urbana W01   
6035  sr:competition:47045        WTA 125K San Sebastian, Spain Women Singles   
6036  sr:competition:47047        WTA 125K San Sebastian, Spain Women Doubles   
6037  sr:competition:47053  WTA 125K Caldas da Rainha, Portugal Women Singles   
6038  sr:competition:47055  WTA 125K Caldas da Rainha, Portugal Women Doubles   

     competition_type compe

2. Count the number of competitions in each category:

In [48]:
sql_query = """Select count(comp.competition_id) as no_of_competitions,
cat.categoryname as category
from 
competition_table comp full join categories_table cat 
on cat.categoryid = comp.categoryid group by cat.categoryname"""

#execute:
mycursor.execute(sql_query)
fetching_data = mycursor.fetchall()

In [49]:
#To fetch column names:
a = mycursor.description

# Create an empty list to store column names
column_names = []

# Loop through each tuple and extract the first item
for col in a:
    column_names.append(col[0])
    
# Create DataFrame
df = pd.DataFrame(fetching_data, columns=column_names)

# Display the DataFrame
print(df)

    no_of_competitions              category
0                   16           Wheelchairs
1                    1             Davis Cup
2                   89               UTR Men
3                    1            United Cup
4                   78             UTR Women
5                   32            Exhibition
6                 2198               ITF Men
7                   11               Legends
8                    1  Billie Jean King Cup
9                 2032             ITF Women
10                 900            Challenger
11                 178              WTA 125K
12                  16               Juniors
13                 223                   ATP
14                   1                  IPTL
15                 255                   WTA
16                   6   Wheelchairs Juniors
17                   1            Hopman Cup


3)	Find all competitions of type 'doubles'

In [50]:
sql_query = """Select * from competition_table where competition_type = 'doubles'"""

#execute:
mycursor.execute(sql_query)
fetching_data = mycursor.fetchall()
fetching_data

[('sr:competition:2557',
  'Wimbledon Men Doubles',
  'sr:competition:2553',
  'doubles',
  'men',
  'sr:category:3'),
 ('sr:competition:2561',
  'Wimbledon Women Doubles',
  'sr:competition:2553',
  'doubles',
  'women',
  'sr:category:6'),
 ('sr:competition:2569',
  'Australian Open Men Doubles',
  'sr:competition:2565',
  'doubles',
  'men',
  'sr:category:3'),
 ('sr:competition:2573',
  'Australian Open Women Doubles',
  'sr:competition:2565',
  'doubles',
  'women',
  'sr:category:6'),
 ('sr:competition:2581',
  'French Open Men Doubles',
  'sr:competition:2577',
  'doubles',
  'men',
  'sr:category:3'),
 ('sr:competition:2585',
  'French Open Women Doubles',
  'sr:competition:2577',
  'doubles',
  'women',
  'sr:category:6'),
 ('sr:competition:2593',
  'US Open Men Doubles',
  'sr:competition:2589',
  'doubles',
  'men',
  'sr:category:3'),
 ('sr:competition:2597',
  'US Open Women Doubles',
  'sr:competition:2589',
  'doubles',
  'women',
  'sr:category:6'),
 ('sr:competition:26

In [51]:
#To fetch column names:
a = mycursor.description

# Create an empty list to store column names
column_names = []

# Loop through each tuple and extract the first item
for col in a:
    column_names.append(col[0])
    
# Create DataFrame
df = pd.DataFrame(fetching_data, columns=column_names)

# Display the DataFrame
print(df)

            competition_id                                   competition_name  \
0      sr:competition:2557                              Wimbledon Men Doubles   
1      sr:competition:2561                            Wimbledon Women Doubles   
2      sr:competition:2569                        Australian Open Men Doubles   
3      sr:competition:2573                      Australian Open Women Doubles   
4      sr:competition:2581                            French Open Men Doubles   
...                    ...                                                ...   
2918  sr:competition:46691  ATP Challenger Nottingham 3, Great Britain Men...   
2919  sr:competition:46955               WTA Sao Paulo, Brazil Women, Doubles   
2920  sr:competition:46961             WTA 125K Porto, Portugal Women Doubles   
2921  sr:competition:47047        WTA 125K San Sebastian, Spain Women Doubles   
2922  sr:competition:47055  WTA 125K Caldas da Rainha, Portugal Women Doubles   

     competition_parent_id 

4)	Get competitions that belong to a specific category (e.g., ITF Men)

In [52]:
sql_query = """Select cat.categoryname, 
comp.competition_name, comp.competition_type, comp.competition_gender from 
categories_table cat full join competition_table comp on
comp.categoryid = cat.categoryid where cat.categoryname = 'ITF Men'"""

#execute:
mycursor.execute(sql_query)
fetching_data = mycursor.fetchall()
fetching_data

[('ITF Men', 'Dominican Republic F2 Men Singles', 'singles', 'men'),
 ('ITF Men', 'Dominican Republic F2 Men Doubles', 'doubles', 'men'),
 ('ITF Men',
  'ITF Men Stuttgart-Stammheim, Germany Men Singles',
  'singles',
  'men'),
 ('ITF Men',
  'ITF Men Stuttgart-Stammheim, Germany Men Doubles',
  'doubles',
  'men'),
 ('ITF Men', 'ITF Men Cherkassy, Ukraine Men Singles', 'singles', 'men'),
 ('ITF Men', 'ITF Men Cherkassy, Ukraine Men Doubles', 'doubles', 'men'),
 ('ITF Men', 'Great Britain F4 Men Singles', 'singles', 'men'),
 ('ITF Men', 'Great Britain F4 Men Doubles', 'doubles', 'men'),
 ('ITF Men', 'ITF Men Vrsar, Croatia Men Singles', 'singles', 'men'),
 ('ITF Men', 'ITF Men Vrsar, Croatia Men Doubles', 'doubles', 'men'),
 ('ITF Men', 'ITF Men Bytom, Poland Men Singles', 'singles', 'men'),
 ('ITF Men', 'ITF Men Bytom, Poland Men Doubles', 'doubles', 'men'),
 ('ITF Men', 'ITF Men Frinton, Great Britain Men Singles', 'singles', 'men'),
 ('ITF Men', 'ITF Men Frinton, Great Britain Men D

In [53]:
#To fetch column names:
a = mycursor.description

# Create an empty list to store column names
column_names = []

# Loop through each tuple and extract the first item
for col in a:
    column_names.append(col[0])
    
# Create DataFrame
df = pd.DataFrame(fetching_data, columns=column_names)

# Display the DataFrame
print(df)

     categoryname                                  competition_name  \
0         ITF Men                 Dominican Republic F2 Men Singles   
1         ITF Men                 Dominican Republic F2 Men Doubles   
2         ITF Men  ITF Men Stuttgart-Stammheim, Germany Men Singles   
3         ITF Men  ITF Men Stuttgart-Stammheim, Germany Men Doubles   
4         ITF Men            ITF Men Cherkassy, Ukraine Men Singles   
...           ...                                               ...   
2193      ITF Men                         ITF Chile F4, Men Singles   
2194      ITF Men                  ITF South Africa F7, Men Singles   
2195      ITF Men                  ITF South Africa F7, Men Doubles   
2196      ITF Men                        ITF Spain F38, Men Singles   
2197      ITF Men                        ITF Spain F38, Men Doubles   

     competition_type competition_gender  
0             singles                men  
1             doubles                men  
2             sing

5)	Identify parent competitions and their sub-competitions

In [54]:
sql_query = """SELECT 
    parent.competition_id AS parent_id,
    parent.competition_name AS parent_name,
    child.competition_id AS sub_competition_id,
    child.competition_name AS sub_competition_name,
    child.competition_type,
    child.competition_gender
FROM 
    competition_table AS parent
inner join
    competition_table AS child
ON 
    parent.competition_id = child.competition_parent_id"""

#execute:
mycursor.execute(sql_query)
fetching_data = mycursor.fetchall()
fetching_data

[('sr:competition:16976',
  'ITF Romania F9, Men Singles',
  'sr:competition:16978',
  'ITF Romania F9, Men Doubles',
  'doubles',
  'men'),
 ('sr:competition:45363',
  'UTR Boca Raton W01',
  'sr:competition:45971',
  'UTR Boca Raton W03',
  'singles',
  'women')]

In [55]:
#To fetch column names:
a = mycursor.description

# Create an empty list to store column names
column_names = []

# Loop through each tuple and extract the first item
for col in a:
    column_names.append(col[0])
    
# Create DataFrame
df = pd.DataFrame(fetching_data, columns=column_names)

# Display the DataFrame
print(df)

              parent_id                  parent_name    sub_competition_id  \
0  sr:competition:16976  ITF Romania F9, Men Singles  sr:competition:16978   
1  sr:competition:45363           UTR Boca Raton W01  sr:competition:45971   

          sub_competition_name competition_type competition_gender  
0  ITF Romania F9, Men Doubles          doubles                men  
1           UTR Boca Raton W03          singles              women  


6)	Analyze the distribution of competition types by category

In [56]:
sql_query = """Select cat.categoryname,comp.competition_type, count(comp.competition_id) as no_of_competitions
from categories_table cat full join competition_table comp on
comp.categoryid = cat.categoryid group by cat.categoryname, comp.competition_type 
order by cat.categoryname, comp.competition_type"""

#execute:
mycursor.execute(sql_query)
fetching_data = mycursor.fetchall()
fetching_data

[('ATP', 'doubles', 107),
 ('ATP', 'mixed', 2),
 ('ATP', 'mixed_doubles', 6),
 ('ATP', 'singles', 108),
 ('Billie Jean King Cup', 'mixed', 1),
 ('Challenger', 'doubles', 449),
 ('Challenger', 'singles', 451),
 ('Davis Cup', 'mixed', 1),
 ('Exhibition', 'doubles', 8),
 ('Exhibition', 'mixed_doubles', 2),
 ('Exhibition', 'singles', 22),
 ('Hopman Cup', 'mixed', 1),
 ('IPTL', 'singles', 1),
 ('ITF Men', 'doubles', 1099),
 ('ITF Men', 'singles', 1099),
 ('ITF Women', 'doubles', 1018),
 ('ITF Women', 'singles', 1014),
 ('Juniors', 'doubles', 8),
 ('Juniors', 'singles', 8),
 ('Legends', 'doubles', 8),
 ('Legends', 'mixed_doubles', 3),
 ('United Cup', 'mixed', 1),
 ('UTR Men', 'singles', 89),
 ('UTR Women', 'singles', 78),
 ('Wheelchairs', 'doubles', 8),
 ('Wheelchairs', 'singles', 8),
 ('Wheelchairs Juniors', 'doubles', 2),
 ('Wheelchairs Juniors', 'singles', 4),
 ('WTA', 'doubles', 127),
 ('WTA', 'singles', 128),
 ('WTA 125K', 'doubles', 89),
 ('WTA 125K', 'singles', 89)]

In [57]:
#To fetch column names:
a = mycursor.description

# Create an empty list to store column names
column_names = []

# Loop through each tuple and extract the first item
for col in a:
    column_names.append(col[0])
    
# Create DataFrame
df = pd.DataFrame(fetching_data, columns=column_names)

# Display the DataFrame
print(df)

            categoryname competition_type  no_of_competitions
0                    ATP          doubles                 107
1                    ATP            mixed                   2
2                    ATP    mixed_doubles                   6
3                    ATP          singles                 108
4   Billie Jean King Cup            mixed                   1
5             Challenger          doubles                 449
6             Challenger          singles                 451
7              Davis Cup            mixed                   1
8             Exhibition          doubles                   8
9             Exhibition    mixed_doubles                   2
10            Exhibition          singles                  22
11            Hopman Cup            mixed                   1
12                  IPTL          singles                   1
13               ITF Men          doubles                1099
14               ITF Men          singles                1099
15      

7)	List all competitions with no parent (top-level competitions)

In [58]:
sql_query = """Select competition_id,competition_name from competition_table where competition_parent_id = 'NaN'"""

#execute:
mycursor.execute(sql_query)
fetching_data = mycursor.fetchall()
fetching_data

[('sr:competition:620', 'Hopman Cup'),
 ('sr:competition:660', 'World Team Cup'),
 ('sr:competition:1207', 'Championship International Series'),
 ('sr:competition:2100', 'Davis Cup'),
 ('sr:competition:2102', 'Billie Jean King Cup'),
 ('sr:competition:12229', 'IPTL'),
 ('sr:competition:15393', 'ITF Men San Jose, Costa Rica Men Singles'),
 ('sr:competition:15395', 'ITF Men San Jose, Costa Rica Men Doubles'),
 ('sr:competition:21028', 'ITF Colombia 02A, Women Singles'),
 ('sr:competition:21228', 'ITF Australia 02B, Women Doubles'),
 ('sr:competition:22334', 'ITF Uzbekistan 01A, Women Singles'),
 ('sr:competition:22763', 'ITF Tunisia 39A, Women Doubles'),
 ('sr:competition:22787', 'ITF Egypt F38, Men Doubles'),
 ('sr:competition:22959', 'ITF USA 17A, Women Singles'),
 ('sr:competition:22997', 'ITF Bosnia & Herzegovina 01A, Women Doubles'),
 ('sr:competition:29204', 'ITF Mexico F13, Men Singles'),
 ('sr:competition:29206', 'ITF Mexico F13, Men Doubles'),
 ('sr:competition:29386', 'ITF Mexi

In [59]:
#To fetch column names:
a = mycursor.description

# Create an empty list to store column names
column_names = []

# Loop through each tuple and extract the first item
for col in a:
    column_names.append(col[0])
    
# Create DataFrame
df = pd.DataFrame(fetching_data, columns=column_names)

# Display the DataFrame
print(df)

           competition_id                   competition_name
0      sr:competition:620                         Hopman Cup
1      sr:competition:660                     World Team Cup
2     sr:competition:1207  Championship International Series
3     sr:competition:2100                          Davis Cup
4     sr:competition:2102               Billie Jean King Cup
..                    ...                                ...
199  sr:competition:46991                     UTR Urbana M01
200  sr:competition:46993                 UTR Tuscaloosa M01
201  sr:competition:46995                 UTR Pazardzhik W01
202  sr:competition:46997                   UTR Budapest W01
203  sr:competition:46999                     UTR Urbana W01

[204 rows x 2 columns]


--COMPLEXES:

1)	List all venues along with their associated complex name

In [60]:
sql_query = """Select comp.complex_name, 
venue.venue_id,venue.venue_name,venue.venue_city_name,venue.venue_country_name,venue.venue_country_code,
venue.venue_timezone from complexes_table comp inner join venues_table venue on comp.complex_id = venue.complex_id"""

#execute:
mycursor.execute(sql_query)
fetching_data = mycursor.fetchall()
fetching_data

[('Nacional',
  'sr:venue:70045',
  'Cancha Central',
  'Santiago',
  'Chile',
  'CHL',
  'America/Santiago'),
 ('Estadio de la Cartuja',
  'sr:venue:74858',
  'Court One',
  'Seville',
  'Spain',
  'ESP',
  'Europe/Madrid'),
 ('Sibur Arena',
  'sr:venue:62153',
  'TC Dynamo',
  'Saint Petersburg',
  'Russia',
  'RUS',
  'Europe/Moscow'),
 ('O2 Arena',
  'sr:venue:77437',
  'Campo 8',
  'Oeiras',
  'Portugal',
  'PRT',
  'Europe/Lisbon'),
 ('Ostravar Arena',
  'sr:venue:55223',
  'Court 2',
  'Ostrava',
  'Czechia',
  'CZE',
  'Europe/Prague'),
 ('Varazdin Arena',
  'sr:venue:77285',
  'Table 8',
  'Varazdin',
  'Croatia',
  'HRV',
  'Europe/Zagreb'),
 ('Palais des sports de Pau',
  'sr:venue:39934',
  'Court 3',
  'Montpellier',
  'France',
  'FRA',
  'Europe/Paris'),
 ('Stade Pierre-de-Coubertin',
  'sr:venue:74197',
  'Court 1',
  'Orleans',
  'France',
  'FRA',
  'Europe/Paris'),
 ('Arena Gripe',
  'sr:venue:72571',
  'Pista 4',
  'Malaga',
  'Spain',
  'ESP',
  'Europe/Madrid'),
 

In [61]:
#To fetch column names:
a = mycursor.description

# Create an empty list to store column names
column_names = []

# Loop through each tuple and extract the first item
for col in a:
    column_names.append(col[0])
    
# Create DataFrame
df = pd.DataFrame(fetching_data, columns=column_names)

# Display the DataFrame
print(df)

                           complex_name        venue_id  \
0                              Nacional  sr:venue:70045   
1                 Estadio de la Cartuja  sr:venue:74858   
2                           Sibur Arena  sr:venue:62153   
3                              O2 Arena  sr:venue:77437   
4                        Ostravar Arena  sr:venue:55223   
..                                  ...             ...   
504           Dan Maskell Tennis Centre  sr:venue:82392   
505  George P Mitchell 40 Tennis Center  sr:venue:82451   
506              National Tennis Center  sr:venue:82479   
507                     Tcja Saint-Malo  sr:venue:82495   
508                     Club Tennis Vic  sr:venue:82499   

                                       venue_name      venue_city_name  \
0                                  Cancha Central             Santiago   
1                                       Court One              Seville   
2                                       TC Dynamo     Saint Petersbur

2)	Count the number of venues in each complex

In [62]:
sql_query = """Select comp.complex_name, count(venue.venue_id) as venue_count
from complexes_table comp left join venues_table venue on comp.complex_id = venue.complex_id 
group by comp.complex_name order by complex_name asc"""

#execute:
mycursor.execute(sql_query)
fetching_data = mycursor.fetchall()
fetching_data

[('A.S.D. Tennis Club Como', 1),
 ('AAJB Tennis', 1),
 ('Abama Tennis Academy', 1),
 ('Academy Zhangjiagang', 1),
 ('Acapulco Princess Mundo Imperial', 0),
 ('Accor Arena', 1),
 ('Ace Tennis Club', 1),
 ('Adani Arena Rockhampton', 1),
 ('AELTC Wimbledon Qualifying and Community Sports Ground', 1),
 ('Akademia Tenisowa Tenis Kozerki', 1),
 ('Alex Metreveli Tennis Club', 1),
 ('All England Lawn Tennis and Croquet Club', 1),
 ('All In Country Club Decines', 1),
 ('Almaty Tennis Center', 1),
 ('Am Rothenbaum', 2),
 ('Amelia Island Plantation Resort', 1),
 ('Amersfoort Lawn Tennis Club', 1),
 ('Anexo Guayaquil Tenis Club', 0),
 ('AO Arena', 1),
 ('Arena Armeets Sofia', 1),
 ('Arena BRB', 1),
 ("Arena Co'Met", 1),
 ('Arena GNP Seguros', 1),
 ('Arena Gripe', 1),
 ('Arena Loire Trelaze', 1),
 ('Arena Montpellier', 0),
 ('Arena Sofia', 1),
 ('Arenes de Metz', 0),
 ('Ariake Colosseum', 1),
 ('Ariake Tennis Forest Park Ariake Colesseum', 1),
 ('Arka Tennis Club', 1),
 ('ASB Tennis Arena', 1),
 ('

In [63]:
#To fetch column names:
a = mycursor.description

# Create an empty list to store column names
column_names = []

# Loop through each tuple and extract the first item
for col in a:
    column_names.append(col[0])
    
# Create DataFrame
df = pd.DataFrame(fetching_data, columns=column_names)

# Display the DataFrame
print(df)

                                      complex_name  venue_count
0                          A.S.D. Tennis Club Como            1
1                                      AAJB Tennis            1
2                             Abama Tennis Academy            1
3                             Academy Zhangjiagang            1
4                 Acapulco Princess Mundo Imperial            0
..                                             ...          ...
629                              XS Tennis Village            1
630                        Yang-Ming Tennis Centre            1
631                        Yokkaichi Tennis Center            1
632                           Yucatan Country Club            1
633  Zayed Sports City International Tennis Centre            1

[634 rows x 2 columns]


3)	Get details of venues in a specific country (e.g., Chile)

In [64]:
sql_query = """Select * from venues_table where venue_country_name = 'Chile'"""

#execute:
mycursor.execute(sql_query)
fetching_data = mycursor.fetchall()
fetching_data

[('sr:complex:705',
  'sr:venue:70045',
  'Cancha Central',
  'Santiago',
  'Chile',
  'CHL',
  'America/Santiago'),
 ('sr:complex:11769',
  'sr:venue:67863',
  'Cancha 3',
  'Temuco',
  'Chile',
  'CHL',
  'America/Santiago'),
 ('sr:complex:33492',
  'sr:venue:35806',
  'Court 5',
  'Vina del Mar',
  'Chile',
  'CHL',
  'America/Santiago'),
 ('sr:complex:50605',
  'sr:venue:76925',
  'Court Jaime Fillol',
  'Santiago',
  'Chile',
  'CHL',
  'America/Santiago'),
 ('sr:complex:57188',
  'sr:venue:57194',
  'Cancha 2',
  'Concepcion',
  'Chile',
  'CHL',
  'America/Santiago'),
 ('sr:complex:62419',
  'sr:venue:57598',
  'Cancha 11',
  'Santiago',
  'Chile',
  'CHL',
  'America/Santiago'),
 ('sr:complex:63409',
  'sr:venue:66901',
  'Cancha 3',
  'Coquimbo',
  'Chile',
  'CHL',
  'America/Santiago'),
 ('sr:complex:69183',
  'sr:venue:77493',
  'Cancha 3',
  'Concepcion',
  'Chile',
  'CHL',
  'America/Santiago'),
 ('sr:complex:73997',
  'sr:venue:74005',
  'Court 3',
  'Antofagasta',
  'C

In [65]:
#To fetch column names:
a = mycursor.description

# Create an empty list to store column names
column_names = []

# Loop through each tuple and extract the first item
for col in a:
    column_names.append(col[0])
    
# Create DataFrame
df = pd.DataFrame(fetching_data, columns=column_names)

# Display the DataFrame
print(df)

          complex_id        venue_id          venue_name venue_city_name  \
0     sr:complex:705  sr:venue:70045      Cancha Central        Santiago   
1   sr:complex:11769  sr:venue:67863            Cancha 3          Temuco   
2   sr:complex:33492  sr:venue:35806             Court 5    Vina del Mar   
3   sr:complex:50605  sr:venue:76925  Court Jaime Fillol        Santiago   
4   sr:complex:57188  sr:venue:57194            Cancha 2      Concepcion   
5   sr:complex:62419  sr:venue:57598           Cancha 11        Santiago   
6   sr:complex:63409  sr:venue:66901            Cancha 3        Coquimbo   
7   sr:complex:69183  sr:venue:77493            Cancha 3      Concepcion   
8   sr:complex:73997  sr:venue:74005             Court 3     Antofagasta   
9   sr:complex:75070  sr:venue:75120          Grandstand          Colina   
10  sr:complex:82223  sr:venue:82231         C4 Skechers        Santiago   

   venue_country_name venue_country_code    venue_timezone  
0               Chile     

4)	Identify all venues and their timezones

In [66]:
sql_query = """SELECT 
    venue_name,
    venue_city_name,
    venue_country_name,
    venue_timezone
FROM 
    venues_table;"""

#execute:
mycursor.execute(sql_query)
fetching_data = mycursor.fetchall()
fetching_data

[('Cancha Central', 'Santiago', 'Chile', 'America/Santiago'),
 ('Court One', 'Seville', 'Spain', 'Europe/Madrid'),
 ('TC Dynamo', 'Saint Petersburg', 'Russia', 'Europe/Moscow'),
 ('Campo 8', 'Oeiras', 'Portugal', 'Europe/Lisbon'),
 ('Court 2', 'Ostrava', 'Czechia', 'Europe/Prague'),
 ('Table 8', 'Varazdin', 'Croatia', 'Europe/Zagreb'),
 ('Court 3', 'Montpellier', 'France', 'Europe/Paris'),
 ('Court 1', 'Orleans', 'France', 'Europe/Paris'),
 ('Pista 4', 'Malaga', 'Spain', 'Europe/Madrid'),
 ('Court 2', 'Sofia', 'Bulgaria', 'Europe/Sofia'),
 ('Court 1', 'Rouen', 'France', 'Europe/Paris'),
 ('Cancha 3', 'Temuco', 'Chile', 'America/Santiago'),
 ('Court 12', 'Lyon', 'France', 'Europe/Paris'),
 ('Court 2', 'Orleans', 'France', 'Europe/Paris'),
 ('Campo Arpex', 'Andria', 'Italy', 'Europe/Rome'),
 ('Court TCSB', 'Saint-Brieuc', 'France', 'Europe/Paris'),
 ('Court Limak Enerji', 'Istanbul', 'Turkiye', 'Europe/Istanbul'),
 ('Court 2', 'Stuttgart', 'Germany', 'Europe/Berlin'),
 ('Centre Court', '

In [67]:
#To fetch column names:
a = mycursor.description

# Create an empty list to store column names
column_names = []

# Loop through each tuple and extract the first item
for col in a:
    column_names.append(col[0])
    
# Create DataFrame
df = pd.DataFrame(fetching_data, columns=column_names)

# Display the DataFrame
print(df)

                                       venue_name      venue_city_name  \
0                                  Cancha Central             Santiago   
1                                       Court One              Seville   
2                                       TC Dynamo     Saint Petersburg   
3                                         Campo 8               Oeiras   
4                                         Court 2              Ostrava   
..                                            ...                  ...   
504            Loughborough University - Court 05         Loughborough   
505  Mitchell Tennis Center - Texas AM - Court 04  College Station, TX   
506   National Tennis Center - Outdoor - Court 06            Bucharest   
507                                       Court 2           Saint Malo   
508                                       Court 3                  Vic   

    venue_country_name    venue_timezone  
0                Chile  America/Santiago  
1                Spain   

5)	Find complexes that have more than one venue

In [68]:
sql_query = """Select comp.complex_name, count(venue.venue_id) as venue_count
from complexes_table comp inner join venues_table venue on comp.complex_id = venue.complex_id 
group by comp.complex_name  having count(venue.venue_id)>1 order by complex_name asc"""

#execute:
mycursor.execute(sql_query)
fetching_data = mycursor.fetchall()
fetching_data

[('Am Rothenbaum', 2),
 ('Baku Tennis Academy', 2),
 ('Carrasco Lawn Tennis Club', 2),
 ('Circolo del Tennis Firenze 1898', 2),
 ('Clube de Ténis do Estoril', 2),
 ('National Tennis Center', 5),
 ('National Tennis Centre', 3),
 ('Nice Lawn Tennis Club', 2),
 ('Tennis Club Triestino', 2),
 ('The Phoenix Country Club', 2),
 ('True Arena Hua Hin', 2)]

In [69]:
#To fetch column names:
a = mycursor.description

# Create an empty list to store column names
column_names = []

# Loop through each tuple and extract the first item
for col in a:
    column_names.append(col[0])
    
# Create DataFrame
df = pd.DataFrame(fetching_data, columns=column_names)

# Display the DataFrame
print(df)

                       complex_name  venue_count
0                     Am Rothenbaum            2
1               Baku Tennis Academy            2
2         Carrasco Lawn Tennis Club            2
3   Circolo del Tennis Firenze 1898            2
4         Clube de Ténis do Estoril            2
5            National Tennis Center            5
6            National Tennis Centre            3
7             Nice Lawn Tennis Club            2
8             Tennis Club Triestino            2
9          The Phoenix Country Club            2
10               True Arena Hua Hin            2


6)	List venues grouped by country

In [70]:
sql_query = """Select venue_country_name, count(*) as venue_count, String_agg (venue_name , ',') as venue_name
from venues_table
group by venue_country_name 
order by venue_country_name """

#execute:
mycursor.execute(sql_query)
fetching_data = mycursor.fetchall()
fetching_data

[('Argentina',
  10,
  'Court Guillermo Villas,Cancha 7,Cancha 3,Cancha Central,Estadio Leonardo Mayer,Cancha 3,Court Central,Club Nautico Hacoaj - Center Court,Cancha 2,Cancha 9'),
 ('Australia',
  12,
  'Court 2,Kia Arena,Centre Court,Show Court 1,Ct 15,West Court,West Park Oval,Court 8,Court 5,Court 2,Centre Court,Court 2'),
 ('Austria',
  9,
  'Next Gen Court,Court Stadtwerke,Show Court,Center Court,Grandstand,Stadium,Court 3,Court 3,Match Court 1'),
 ('Bahrain', 1, 'Court 2'),
 ('Belgium', 2, 'Court 3,Court 1'),
 ('Bolivia', 2, 'Cancha Central,Cancha 2'),
 ('Bosnia & Herzegovina', 2, 'Court 1,Court 2'),
 ('Brazil',
  11,
  'Central,Indoors,Indoor 4,Quadra 2,Court 2,Court 4,Indoor 4,Indoor 12,Quadra 4,Indoor Court,Indoor 1'),
 ('Bulgaria',
  4,
  'Court 2,Court 3,Court 3,Tennis Courts of the SW University Neofit Rilski - Court 2'),
 ('Canada',
  8,
  'Cashmere Grandstand,Rogers Court,Court Central,Court 1,Polygon Court,Indoor 2,Centre Marcel Dionne,Court 3'),
 ('Chile',
  11,
  'Ca

In [71]:
#To fetch column names:
a = mycursor.description

# Create an empty list to store column names
column_names = []

# Loop through each tuple and extract the first item
for col in a:
    column_names.append(col[0])
    
# Create DataFrame
df = pd.DataFrame(fetching_data, columns=column_names)

# Display the DataFrame
print(df)

      venue_country_name  venue_count  \
0              Argentina           10   
1              Australia           12   
2                Austria            9   
3                Bahrain            1   
4                Belgium            2   
..                   ...          ...   
62               Ukraine            1   
63  United Arab Emirates            3   
64               Uruguay            2   
65                   USA           63   
66            Uzbekistan            4   

                                           venue_name  
0   Court Guillermo Villas,Cancha 7,Cancha 3,Canch...  
1   Court 2,Kia Arena,Centre Court,Show Court 1,Ct...  
2   Next Gen Court,Court Stadtwerke,Show Court,Cen...  
3                                             Court 2  
4                                     Court 3,Court 1  
..                                                ...  
62                                           Court 11  
63   Court 2,ADCB Court 1,Atlantis The Palm - Court 4  
64 

7)	Find all venues for a specific complex (e.g., Nacional)

In [72]:
sql_query = """Select 
  comp.complex_name,
  venue.venue_id,venue.venue_name,venue.venue_city_name,venue.venue_country_name,  venue.venue_country_code,  venue.venue_timezone 
  from complexes_table comp inner join venues_table venue 
  on comp.complex_id = venue.complex_id 
  where complex_name = 'Nacional'"""

#execute:
mycursor.execute(sql_query)
fetching_data = mycursor.fetchall()
fetching_data

[('Nacional',
  'sr:venue:70045',
  'Cancha Central',
  'Santiago',
  'Chile',
  'CHL',
  'America/Santiago')]

In [73]:
#To fetch column names:
a = mycursor.description

# Create an empty list to store column names
column_names = []

# Loop through each tuple and extract the first item
for col in a:
    column_names.append(col[0])
    
# Create DataFrame
df = pd.DataFrame(fetching_data, columns=column_names)

# Display the DataFrame
print(df)

  complex_name        venue_id      venue_name venue_city_name  \
0     Nacional  sr:venue:70045  Cancha Central        Santiago   

  venue_country_name venue_country_code    venue_timezone  
0              Chile                CHL  America/Santiago  


--COMPETITOR RANKINGS:

1)	Get all competitors with their rank and points.

In [74]:
sql_query = """Select comp.competitor_name,comp_rank.competitor_rank,comp_rank.competitor_rank_points 
from competitors_table comp inner join competitor_rankings_table comp_rank
on comp_rank.competitor_id = comp.competitor_id
order by competitor_rank asc"""

#execute:
mycursor.execute(sql_query)
fetching_data = mycursor.fetchall()
fetching_data

[('Pavic, Mate', 1, 9950),
 ('Siniakova, Katerina', 1, 10665),
 ('Arevalo-Gonzalez, Marcelo', 1, 9950),
 ('Townsend, Taylor', 2, 8620),
 ('Routliffe, Erin', 3, 8125),
 ('Heliovaara, Harri', 3, 7590),
 ('Patten, Henry', 4, 7590),
 ('Ostapenko, Jelena', 4, 6950),
 ('Krawietz, Kevin', 5, 6180),
 ('Dabrowski, Gabriela', 5, 6385),
 ('Putz, Tim', 6, 6090),
 ('Paolini, Jasmine', 6, 5465),
 ('Errani, Sara', 6, 5465),
 ('Granollers, Marcel', 7, 5675),
 ('Zeballos, Horacio', 8, 5675),
 ('Muhammad, Asia', 8, 4690),
 ('Kichenok, Lyudmyla', 9, 4630),
 ('Vavassori, Andrea', 9, 5580),
 ('Bolelli, Simone', 10, 5520),
 ('Zhang, Shuai', 10, 4523),
 ('Thompson, Jordan', 11, 5460),
 ('Hsieh, Su-Wei', 11, 4521),
 ('Kudermetova, Veronika', 12, 4365),
 ('Purcell, Max', 12, 4710),
 ('Mektic, Nikola', 13, 4700),
 ('Shnaider, Diana', 13, 4356),
 ('Glasspool, Lloyd', 14, 4460),
 ('Krawczyk, Desirae', 14, 4133),
 ('Khromacheva, Irina', 15, 3968),
 ('Cash, Julian', 15, 4325),
 ('Chan, Hao-Ching', 16, 3945),
 ('Ven

In [75]:
#To fetch column names:
a = mycursor.description

# Create an empty list to store column names
column_names = []

# Loop through each tuple and extract the first item
for col in a:
    column_names.append(col[0])
    
# Create DataFrame
df = pd.DataFrame(fetching_data, columns=column_names)

# Display the DataFrame
print(df)

               competitor_name  competitor_rank  competitor_rank_points
0                  Pavic, Mate                1                    9950
1          Siniakova, Katerina                1                   10665
2    Arevalo-Gonzalez, Marcelo                1                    9950
3             Townsend, Taylor                2                    8620
4              Routliffe, Erin                3                    8125
..                         ...              ...                     ...
995   Ambrogi, Luciano Emanuel              498                     108
996               Kirkin, Ergi              499                     108
997             Walker, Brandy              499                     128
998       Guillen Meza, Alvaro              500                     108
999                 Lene, Emma              500                     126

[1000 rows x 3 columns]


2)	Find competitors ranked in the top 5

In [76]:
sql_query = """Select comp.competitor_name,comp_rank.competitor_rank,comp_rank.competitor_rank_points 
from competitors_table comp inner join competitor_rankings_table comp_rank
on comp_rank.competitor_id = comp.competitor_id
where competitor_rank <=5 order by competitor_rank asc"""

#execute:
mycursor.execute(sql_query)
fetching_data = mycursor.fetchall()
fetching_data

[('Pavic, Mate', 1, 9950),
 ('Arevalo-Gonzalez, Marcelo', 1, 9950),
 ('Siniakova, Katerina', 1, 10665),
 ('Townsend, Taylor', 2, 8620),
 ('Routliffe, Erin', 3, 8125),
 ('Heliovaara, Harri', 3, 7590),
 ('Patten, Henry', 4, 7590),
 ('Ostapenko, Jelena', 4, 6950),
 ('Krawietz, Kevin', 5, 6180),
 ('Dabrowski, Gabriela', 5, 6385)]

In [77]:
#To fetch column names:
a = mycursor.description

# Create an empty list to store column names
column_names = []

# Loop through each tuple and extract the first item
for col in a:
    column_names.append(col[0])
    
# Create DataFrame
df = pd.DataFrame(fetching_data, columns=column_names)

# Display the DataFrame
print(df)

             competitor_name  competitor_rank  competitor_rank_points
0                Pavic, Mate                1                    9950
1  Arevalo-Gonzalez, Marcelo                1                    9950
2        Siniakova, Katerina                1                   10665
3           Townsend, Taylor                2                    8620
4            Routliffe, Erin                3                    8125
5          Heliovaara, Harri                3                    7590
6              Patten, Henry                4                    7590
7          Ostapenko, Jelena                4                    6950
8            Krawietz, Kevin                5                    6180
9        Dabrowski, Gabriela                5                    6385


3)	List competitors with no rank movement (stable rank)

In [78]:
sql_query = """Select comp.competitor_name,
comp_rank.competitor_rank,comp_rank.competitor_rank_points,comp_rank.competitor_rank_movement
from competitors_table comp inner join competitor_rankings_table comp_rank
on comp_rank.competitor_id = comp.competitor_id where competitor_rank_movement = 0"""

#execute:
mycursor.execute(sql_query)
fetching_data = mycursor.fetchall()
fetching_data

[('Pavic, Mate', 1, 9950, 0),
 ('Arevalo-Gonzalez, Marcelo', 1, 9950, 0),
 ('Heliovaara, Harri', 3, 7590, 0),
 ('Patten, Henry', 4, 7590, 0),
 ('Purcell, Max', 12, 4710, 0),
 ('Mektic, Nikola', 13, 4700, 0),
 ('Glasspool, Lloyd', 14, 4460, 0),
 ('Cash, Julian', 15, 4325, 0),
 ('Venus, Michael', 16, 4000, 0),
 ('Skupski, Neal', 17, 3820, 0),
 ('Dodig, Ivan', 55, 1635, 0),
 ('Miedler, Lucas', 56, 1635, 0),
 ('Schnaitter, Jakob', 59, 1570, 0),
 ('Wallner, Mark', 59, 1570, 0),
 ('Jacq, Gregoire', 63, 1435, 0),
 ('Reyes-Varela, Miguel', 71, 1320, 0),
 ('Krajicek, Austin', 73, 1260, 0),
 ('Bollipalli, Rithvik Choudary', 76, 1185, 0),
 ('Matuszewski, Piotr', 77, 1168, 0),
 ('McDonald, Mackenzie', 81, 1120, 0),
 ('Michelsen, Alex', 82, 1095, 0),
 ('Trhac, Patrik', 94, 872, 0),
 ('Harper, Cleeve', 97, 843, 0),
 ('Zormann, Marcelo', 103, 808, 0),
 ('Soto, Matias', 106, 793, 0),
 ('Blancaneaux, Geoffrey', 123, 674, 0),
 ('Merino, Alexander', 146, 551, 0),
 ('Negritu, Christoph', 147, 551, 0),
 ('

In [79]:
#To fetch column names:
a = mycursor.description

# Create an empty list to store column names
column_names = []

# Loop through each tuple and extract the first item
for col in a:
    column_names.append(col[0])
    
# Create DataFrame
df = pd.DataFrame(fetching_data, columns=column_names)

# Display the DataFrame
print(df)

              competitor_name  competitor_rank  competitor_rank_points  \
0                 Pavic, Mate                1                    9950   
1   Arevalo-Gonzalez, Marcelo                1                    9950   
2           Heliovaara, Harri                3                    7590   
3               Patten, Henry                4                    7590   
4                Purcell, Max               12                    4710   
..                        ...              ...                     ...   
93         Chaudhari, Vaidehi              351                     215   
94            In-Albon, Ylena              352                     214   
95        Prisacariu, Andreea              369                     196   
96    Perez Alarcon, Lucciana              377                     186   
97            Pace, Francesca              438                     151   

    competitor_rank_movement  
0                          0  
1                          0  
2                 

4)	Get the total points of competitors from a specific country (e.g., Croatia)

In [80]:
sql_query = """Select sum (comp_rank.competitor_rank_points) as croatia_country_total_points,
String_agg(comp.competitor_name, ',') as competitiors
from competitors_table comp inner join competitor_rankings_table comp_rank
on comp_rank.competitor_id = comp.competitor_id where comp.competitor_country = 'Croatia'"""

#execute:
mycursor.execute(sql_query)
fetching_data = mycursor.fetchall()
fetching_data

[(17452,
  'Pavic, Mate,Mektic, Nikola,Dodig, Ivan,Poljicak, Mili,Wuerth, Tara,Drazic, Mariana,Marcinko, Petra,Martic, Petra')]

In [81]:
#To fetch column names:
a = mycursor.description

# Create an empty list to store column names
column_names = []

# Loop through each tuple and extract the first item
for col in a:
    column_names.append(col[0])
    
# Create DataFrame
df = pd.DataFrame(fetching_data, columns=column_names)

# Display the DataFrame
print(df)

   croatia_country_total_points  \
0                         17452   

                                        competitiors  
0  Pavic, Mate,Mektic, Nikola,Dodig, Ivan,Poljica...  


5)	Count the number of competitors per country

In [82]:
sql_query = """Select count(*) as no_of_competitiors,competitor_country from competitors_table 
group by competitor_country 
order by competitor_country asc"""

#execute:
mycursor.execute(sql_query)
fetching_data = mycursor.fetchall()
fetching_data

[(1, 'Algeria'),
 (1, 'Antigua and Barbuda'),
 (32, 'Argentina'),
 (1, 'Armenia'),
 (47, 'Australia'),
 (13, 'Austria'),
 (1, 'Belarus'),
 (11, 'Belgium'),
 (5, 'Bolivia'),
 (2, 'Bosnia & Herzegovina'),
 (23, 'Brazil'),
 (7, 'Bulgaria'),
 (1, 'Burundi'),
 (15, 'Canada'),
 (5, 'Chile'),
 (31, 'China'),
 (13, 'Chinese Taipei'),
 (9, 'Colombia'),
 (1, 'Costa Rica'),
 (8, 'Croatia'),
 (1, 'Cyprus'),
 (38, 'Czechia'),
 (3, 'Denmark'),
 (5, 'Ecuador'),
 (1, 'Egypt'),
 (1, 'El Salvador'),
 (3, 'Estonia'),
 (5, 'Finland'),
 (54, 'France'),
 (5, 'Georgia'),
 (31, 'Germany'),
 (51, 'Great Britain'),
 (9, 'Greece'),
 (2, 'Hong Kong, China'),
 (5, 'Hungary'),
 (30, 'India'),
 (2, 'Indonesia'),
 (1, 'Ireland'),
 (3, 'Israel'),
 (40, 'Italy'),
 (1, 'Jamaica'),
 (58, 'Japan'),
 (1, 'Jordan'),
 (9, 'Kazakhstan'),
 (1, 'Kenya'),
 (5, 'Latvia'),
 (2, 'Lebanon'),
 (1, 'Lithuania'),
 (1, 'Luxembourg'),
 (11, 'Mexico'),
 (2, 'Monaco'),
 (1, 'Morocco'),
 (29, 'Netherlands'),
 (61, 'Neutral'),
 (9, 'New Zeal

In [83]:
#To fetch column names:
a = mycursor.description

# Create an empty list to store column names
column_names = []

# Loop through each tuple and extract the first item
for col in a:
    column_names.append(col[0])
    
# Create DataFrame
df = pd.DataFrame(fetching_data, columns=column_names)

# Display the DataFrame
print(df)

    no_of_competitiors   competitor_country
0                    1              Algeria
1                    1  Antigua and Barbuda
2                   32            Argentina
3                    1              Armenia
4                   47            Australia
..                 ...                  ...
76                   3              Uruguay
77                 103                  USA
78                   1           Uzbekistan
79                   4            Venezuela
80                   2             Zimbabwe

[81 rows x 2 columns]


6)	Find competitors with the highest points in the current week

In [84]:
sql_query = """SELECT comp.competitor_name,comp_rank.competitor_rank_points FROM competitors_table comp JOIN 
    competitor_rankings_table comp_rank
    ON comp.competitor_id = comp_rank.competitor_id 
	where  comp_rank.competitor_rank_points = (SELECT MAX(competitor_rank_points)
        FROM competitor_rankings_table)"""

#execute:
mycursor.execute(sql_query)
fetching_data = mycursor.fetchall()
fetching_data

[('Siniakova, Katerina', 10665)]

In [85]:
#To fetch column names:
a = mycursor.description

# Create an empty list to store column names
column_names = []

# Loop through each tuple and extract the first item
for col in a:
    column_names.append(col[0])
    
# Create DataFrame
df = pd.DataFrame(fetching_data, columns=column_names)

# Display the DataFrame
print(df)

       competitor_name  competitor_rank_points
0  Siniakova, Katerina                   10665


In [86]:
mycursor.close()
my_db_connection.close()